In [1]:
import numpy as np 

In [2]:
A=np.random.rand(3,5)

In [3]:
A

array([[0.58664933, 0.86268652, 0.00792486, 0.30862384, 0.57644987],
       [0.25641696, 0.12498571, 0.3261824 , 0.76215331, 0.94327737],
       [0.90976249, 0.80115775, 0.93973165, 0.58713391, 0.40636883]])

In [4]:
np.linalg.svd(A)

(array([[-0.49303626,  0.15010851, -0.85696131],
        [-0.47959239, -0.86872067,  0.12375597],
        [-0.72588318,  0.4720083 ,  0.50030169]]),
 array([2.22562278, 0.82440106, 0.63549382]),
 array([[-0.48193086, -0.47933768, -0.3785355 , -0.42409536, -0.46349976],
        [ 0.35749807,  0.4840751 ,  0.19576556, -0.41076995, -0.65636143],
        [-0.02493642, -0.5082672 ,  0.79265138,  0.19447304, -0.27372733],
        [-0.4758927 ,  0.37204344, -0.08774702,  0.66645533, -0.42807425],
        [-0.64252789,  0.37303461,  0.42707256, -0.41161717,  0.31013453]]))

In [5]:
import scipy
import scipy.linalg

In [6]:
from datetime import datetime

In [136]:
M=np.random.rand(3,3)

In [137]:
M

array([[0.10252966, 0.66596766, 0.95956141],
       [0.31412641, 0.49532295, 0.30373635],
       [0.18210204, 0.86132001, 0.85865439]])

In [138]:
P,L,U=scipy.linalg.lu(M)

## backward & forward substitution

In [139]:
def timer(func):
    def layer(*args,**kwargs):
        #print(args)
        def count(*args,**kwargs):
            start=datetime.now()
            res=func(*args,**kwargs)
            print(datetime.now()-start)
            return res
        return count(*args,**kwargs)
    return layer

In [204]:
@timer
def forward(L,b):
    d=np.shape(L)[0]
    Y=np.zeros(d)
    for i in range(d):
        l=L[i]
        Y[i]=b[i]
        for c in range(i):
            Y[i]-=Y[c]*l[c]
        Y[i]=Y[i]/l[i]
    return Y

In [205]:
@timer
# optimize
def forward_o(L,b):
    d=np.shape(L)[0]
    Y=np.copy(b)
    for i in range(d):
        l=L[i]
        Y[i]=Y[i]/l[i]
        Y[i+1:]-=Y[i]*L[i+1:,i]
    return Y

In [209]:
d=400
M=np.random.rand(d,d)
b=np.random.rand(d)
P,L,U=scipy.linalg.lu(M)

In [210]:
_=forward_o(L,b)

0:00:00.006988


In [211]:
_=forward(L,b)

0:00:00.098731


In [115]:
#@timer
def backward(U,b):
    d=np.shape(U)[0]
    Y=np.zeros(d)
    for i in range(d-1,-1,-1):
        #i=d-m-1
        l=U[i]
        Y[i]=b[i]
        for c in range(d-1,i,-1):
            Y[i]-=Y[c]*l[c]
        Y[i]=Y[i]/l[i]
    return Y

In [40]:
b=np.random.rand(np.shape(U)[0])

In [41]:
Y=backward(U,b)
print(Y)

0:00:00.000992
[ 0.71387643  3.88181708 -1.69705465 -4.74923419 -3.15346138  2.40569473
  4.34176056 -2.30292469  3.93530006  1.07921956]


In [42]:
np.dot(np.linalg.inv(U),b)

array([ 0.71387643,  3.88181708, -1.69705465, -4.74923419, -3.15346138,
        2.40569473,  4.34176056, -2.30292469,  3.93530006,  1.07921956])

In [130]:
forward(L,b)

array([ 0.64573111,  0.74871002,  0.5136383 , -0.91968758])

In [132]:
forward_o(L,b)

ValueError: operands could not be broadcast together with shapes (289,) (3,) (289,) 

In [44]:
np.dot(np.linalg.inv(L),b)

array([ 0.79946411,  0.00691092, -0.04882792,  0.06954646,  0.47180628,
        0.34553958, -0.43083605, -0.40760726,  0.41290504,  0.30060531])

## time consuming

In [45]:
@timer
def inv(L,b):
    return np.dot(np.linalg.inv(L),b)

In [57]:
d=4
M=np.random.rand(d,d)
b=np.random.rand(d)
P,L,U=scipy.linalg.lu(M)

In [110]:
pb=np.dot(np.linalg.inv(P),b)
Y=forward(L,pb)
print(np.allclose(np.dot(L,Y),pb))
X=backward(U,Y)
print(np.dot(U,X))


0:00:00
True
0:00:00
[ 0.47894067 -0.06820787  0.18863266  0.474001  ]


## 置换矩阵
$P$为$LU$分解时的置换矩阵，为了防止因为主元为零（在数值计算中主元值很小）而交换矩阵两行的变换。如果我们想写出更普适的 $LU$ 分解式的话，必须把行交换情况考虑进去，即：$PA$ 先用行交换使得主元位置不为 0，行顺序正确。其后再用 $LU$ 分解。

In [92]:
print(np.dot(P,np.dot(L,U)))
print(M)
print(np.allclose(np.dot(P,np.dot(L,U)),M))

[[0.76485184 0.98605722 0.38653124 0.60066463]
 [0.23076152 0.77537277 0.16908712 0.2003375 ]
 [0.82089899 0.69583508 0.0227756  0.23570014]
 [0.80933263 0.29578319 0.38680694 0.9643746 ]]
[[0.76485184 0.98605722 0.38653124 0.60066463]
 [0.23076152 0.77537277 0.16908712 0.2003375 ]
 [0.82089899 0.69583508 0.0227756  0.23570014]
 [0.80933263 0.29578319 0.38680694 0.9643746 ]]
True


In [105]:
print((np.dot(P,np.dot(M,X))-b))

[-2.81469670e-01  1.11022302e-16  5.09265546e-01 -2.27795876e-01]


In [104]:
print(np.allclose(np.dot(P,np.dot(M,X)),b,rtol=0.1))

False


In [106]:
res=inv(M,b)
print(np.allclose(np.dot(M,res),b))
print((np.dot(M,res)-b))


0:00:00
True
[-2.22044605e-16 -1.11022302e-16 -2.22044605e-16 -1.11022302e-15]


## 精度低

In [120]:
@timer
def lu_s(M,b):
    P,L,U=scipy.linalg.lu(M)
    #b=np.dot(np.linalg.inv(P),b)
    Y=forward(L,b)#np.dot(np.linalg.inv(L),b)
    X=backward(U,Y)#np.dot(np.linalg.inv(U),Y)

In [124]:
for d in range(10,400,40):
    print('~~~~~~~~~~~~d='+str(d)+'~~~~~~~~~~~')
    #d=400
    M=np.random.rand(d,d)
    b=np.random.rand(d)
    for i in range(3):
        lu_s(M,b)
        _=inv(M,b)
#lu_s(M,b)

~~~~~~~~~~~~d=10~~~~~~~~~~~
0:00:00.001001
0:00:00.000996
0:00:00.000996
0:00:00
0:00:00.000998
0:00:00.001001
~~~~~~~~~~~~d=50~~~~~~~~~~~
0:00:00.010965
0:00:00.008974
0:00:00.007979
0:00:00.002992
0:00:00.007653
0:00:00.001995
~~~~~~~~~~~~d=90~~~~~~~~~~~
0:00:00.024973
0:00:00.023937
0:00:00.020945
0:00:00.109218
0:00:00.038412
0:00:00.030919
~~~~~~~~~~~~d=130~~~~~~~~~~~
0:00:00.039891
0:00:00.079787
0:00:00.039895
0:00:00.075798
0:00:00.042885
0:00:00.104719
~~~~~~~~~~~~d=170~~~~~~~~~~~
0:00:00.102724
0:00:00.089761
0:00:00.086765
0:00:00.070809
0:00:00.080783
0:00:00.091755
~~~~~~~~~~~~d=210~~~~~~~~~~~
0:00:00.137399
0:00:00.174533
0:00:00.182827
0:00:00.155520
0:00:00.161568
0:00:00.190008
~~~~~~~~~~~~d=250~~~~~~~~~~~
0:00:00.241354
0:00:00.183510
0:00:00.234373
0:00:00.159575
0:00:00.189472
0:00:00.234885
~~~~~~~~~~~~d=290~~~~~~~~~~~
0:00:00.226408
0:00:00.307213
0:00:00.239362
0:00:00.244347
0:00:00.209441
0:00:00.388505
~~~~~~~~~~~~d=330~~~~~~~~~~~
0:00:00.436097
0:00:00.307180

In [125]:
    d=290
    M=np.random.rand(d,d)
    b=np.random.rand(d)
    for i in range(9):
        lu_s(M,b)
        _=inv(M,b)

0:00:00.165558
0:00:00.285743
0:00:00.246865
0:00:00.211945
0:00:00.236505
0:00:00.381976
0:00:00.244873
0:00:00.265292
0:00:00.261301
0:00:00.251328
0:00:00.248599
0:00:00.205449
0:00:00.217003
0:00:00.298717
0:00:00.251682
0:00:00.201459
0:00:00.376002
0:00:00.259308
